In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import glob
import os
import math
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import preprocessing
pd.set_option('display.max_columns', 200)

In [ ]:
train_df = pd.read_csv('train.csv')

# 基礎確認

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df["workclass"].value_counts()

In [ ]:
train_df["fnlwgt"].value_counts()

In [ ]:
train_df['fnlwgt'] = train_df['fnlwgt'].astype(str)
train_df['fnlwgt'].value_counts()

In [ ]:
train_df["education"].value_counts()

In [ ]:
train_df["marital-status"].value_counts()

In [ ]:
train_df["occupation"].value_counts()

In [ ]:
train_df["relationship"].value_counts()

In [ ]:
train_df["race"].value_counts()

In [ ]:
train_df["native-country"].value_counts()

# 前処理

In [ ]:
train_df.shape

In [ ]:
train_df['fnlwgt']=train_df['fnlwgt'].astype(str)

In [ ]:
train_df.dtypes

In [ ]:
test_df = pd.read_csv('test.csv')
test_df['fnlwgt']=test_df['fnlwgt'].astype(str)

In [ ]:
test_df.dtypes

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
concat_df = pd.concat([train_df.iloc[:, 0:12], test_df])
concat_df.head()

In [ ]:
concat_df.dtypes

In [ ]:
concat_df['fnlwgt']

In [ ]:
for val in concat_df['fnlwgt'].unique():
    print(f':{val}:')

In [ ]:
concat_df = pd.get_dummies(concat_df)

In [ ]:
train_rev_df = concat_df.iloc[0:11900, :]
train_rev_df.head()

In [ ]:
train_rev_df.shape

In [ ]:
train_rev_df['Y'] = train_df['Y']
train_rev_df.head()

In [ ]:
test_rev_df = concat_df.iloc[11900:, :]
test_rev_df.head()

In [ ]:
train_rev_df.shape

In [ ]:
# 0がやや多いので比率は調整する必要あり
train_rev_df['Y'].value_counts()

In [ ]:
train_rev_df.iloc[:, :-1].head()

# 学習

In [ ]:
import lightgbm
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_rev_df.iloc[:, 1:-1], train_rev_df.iloc[:, -1], test_size=0.3, random_state=42)

In [ ]:
print(f'x_train:shape: {x_train.shape}, x_val:shape: {x_val.shape}, y_train:{y_train.shape}, y_val:{y_val.shape}')

In [ ]:
x_train.head()

In [ ]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=3)
lgb_clf.fit(x_train, y_train)

In [ ]:
y_pred = lgb_clf.predict(x_val)
accuracy_score(y_val, y_pred)

In [ ]:
y_pred = lgb_clf.predict(test_rev_df.iloc[:, 1:])
y_pred

In [ ]:
len(y_pred)

In [ ]:
submit_df = pd.read_csv('sample_submit.csv', header=None)
submit_df.head()

In [ ]:
submit_df['2'] = y_pred
submit_df.head()

In [ ]:
submit_df = submit_df.drop(submit_df.columns[[1]], axis=1)

In [ ]:
submit_df.to_csv("submit3.csv", sep=",", header=None, index=0)

# サンプリング数の変更

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
x_train.shape

In [ ]:
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_sample(x_train, y_train)

In [ ]:
x_train_resampled.shape, y_train_resampled.shape

In [ ]:
y_train_resampled.value_counts()

In [ ]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=20)
lgb_clf.fit(x_train_resampled, y_train_resampled)
y_pred = lgb_clf.predict(x_val)
accuracy_score(y_val, y_pred)

In [ ]:
y_pred = lgb_clf.predict(test_rev_df.iloc[:, 1:])
y_pred

In [ ]:
submit_df = pd.read_csv('sample_submit.csv', header=None)
submit_df['2'] = y_pred
submit_df = submit_df.drop(submit_df.columns[[1]], axis=1)

In [ ]:
submit_df.to_csv("submit_smote.csv", sep=",", header=None, index=0)